# 📁 010_bronze/sharepoint Documentation

This folder contains a series of identical ingestion notebooks, one for each specific wholesaler data source (SharePoint folder).

The purpose of these notebooks is to load raw data from the Landing Zone, apply a source-specific column mapping, and persist the data in the Bronze Layer using a **Golden Template Schema**. This ensures that highly heterogeneous source files are standardized into a unified structure early in the pipeline.

## 🎯 **Source and Target**

* **Source Catalog/Schema:** `d_dnst_000_landing`.volumes (Specific `sellout_data/` paths)
* **Target Catalog/Schema:** `d_dnst_010_bronze`.sharepoint
* **Operation Type:** `overwrite` (Each notebook overwrites its own dedicated table)

## ⚙️ **Process Flow (Golden Template Mapping)**

All notebooks follow the exact same procedure, customized only by a configuration dictionary (`config`) specific to the wholesaler and file version:

1.  **Configuration Definition:** A `config` dictionary is defined, which contains:
    * `folder_name`: The specific folder name within the Landing Zone.
    * `column_mapping`: A map of **Target Column Name** $\rightarrow$ **Source Column Name**. This is the core logic that harmonizes schema differences.
        * Columns that are constant (like `wholesaler_name` and `wholesaler_id`) are populated using Spark literals (`F.lit()`).
        * Columns that are not present in the source are explicitly set to `None`.
2.  **Execution:** The shared function `process_wholesaler_csv_files` is called, which handles:
    * Reading the CSV files from the defined `base_volume_path`.
    * Applying the `column_mapping` to project the raw data onto the predefined `golden_template_schema`.
    * Persisting the result to the dedicated target table (`d_dnst_010_bronze.sharepoint.[wholesaler_name]`).

## 📄 **Output Tables (Example)**

Each notebook creates a dedicated table for its wholesaler data source. These tables share the same schema but contain distinct data based on the source folder.

| Example Output Table (`target_table`) | Notebook Source | Wholesaler ID |
| :--- | :--- | :--- |
| `agora_culinair_van_hout_5525957_v0` | Agora Culinair Van Hout | `5525957` |
| `agora_culinair_vlb_5525741_v0` | Agora Culinair VLB | `5525741` |
| *... and many others* | | |

The tables in this schema serve as the raw, standardized inputs for the `01_prepare_sellout_consolidated` Silver notebook.